In [ ]:
import math

import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# 1. We load some data

* If you are unfamiliar with the Iris Flower dataset, check it here: https://en.wikipedia.org/wiki/Iris_flower_data_set . In Machine Learning it is so popular that it becomes already pre-loaded in SciKit Learn, that's why I can import it using `from sklearn.datasets import load_iris`

* Once the data is loaded in "iris", we discard one of the three classes, to make a binary classification (so we can better explain and plot Logistic Regression's inner workings. We display the number of records we will work with after we discard one class (Feel free to change the class discarded: it can be 0, 1 or 2).
* After we have just 2 classes, we split the data in training and testing. Training will be used to build the Logistic Regression model, and testing to test the model and see how accurate it is. Feel free to change the % of data used for testing - it is set to 0.8 (80%)

In [ ]:
# First we download some data
iris_data = load_iris(return_X_y=True, as_frame=True)
iris = iris_data[0]
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
iris.columns = features
iris['type'] = iris_data[1] # We add the target (class) column to the features dataframe
del iris_data # using del we delete some data

In [ ]:
# For this example, we take just 2 classes to perform some binary classification
# We have 3 classes [0, 1 and 2], so just choose 2 of them to classify from.
# We are discarding class 0 (therefore classifying 1 vs 2), but feel free to change it to:
# CLASS_DISCARDED = 1 (we'd classify between 0 and 2)
# CLASS_DISCARDED = 2 (we'd classify between 0 and 1)
CLASS_DISCARDED = 0
iris_binary = iris[iris['type'] != CLASS_DISCARDED]

print('We will work with',len(iris_binary),'records')

In [ ]:
PERCENTAGE_SAMPLES_USED_FOR_TESTING = 0.8

train, test = train_test_split(iris_binary, test_size=PERCENTAGE_SAMPLES_USED_FOR_TESTING)
print('Using', len(train),'samples for training the model and',len(test),'samples for testing it later.')

# 2. We fit and test our Logistic Regression classifier

* You can check all the parameters that Scikit-Learn allows to tune our Logistic Regression model: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

* Once we create the classifier, we can fit it (build the "best fit" sigmoid function as we saw in the slides), using the training data we separated in the previous section
* You can change the number of max iterations for the optimisation process to find the best fit sigmoid (usually this is a maximum likelihood process, but can be some other too)
* Once the model is fit, we display the intercept and the coefficients of the 4 features: ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
* We also print the expit function that can be used to transform the odds into class probabilities (i.e. to move the data from the logit chart to the sigmoid function chart)
* Then we use the fit model to predict the data we left aside for testing (test data), and we print the predictions.
* Those predictions of one class or another are based on whether the probability we obtained in the expit function is over 0.5 or not. We can change that threshold to improve the accuracy of our model with future data.
* For that, we plot the data in scatter plots, differentiating the two classes, and ask you to select a better threshold than 0.5.

In [ ]:
# First we say to Scikit-Learn the type of classification model we want to make

# Change the number of iterations that the Maximum Likehood Estimator has to converge. Lower numbers should affect
# performance
MAX_ITERATIONS_MLE = 10

classifier = LogisticRegression(
    random_state=0,  # Shuffling the data
    solver='liblinear',  # No need to use Gradient Descent or anything fancy to find the MLE for such small dataset
    penalty='l2',  # We apply L2 regularisation (same regularisation we apply in regression models)
    n_jobs=1,  # Use one processor in this machine for the MLE process
    max_iter=10,  # Max number of iterations for the MLE process
)
# Note: MLE is the Maximum Likelihood Estimator

In [ ]:
# Now we fit our model (the sigmoid function we saw in the slides) using the training data.
# The function classifier.fit(X, y) expects:
# 1. A matrix X (or array of arrays) containing one row per data record, each containing a value for each feature: in our case 4 features.
# 2. An array with the class values that is expected for each data record. 
classifier.fit(train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], train['type'])

# Number of iterations until the MLE process converged.
print('The classifier was correctly fit given the',len(train), 'records of training data.')
print('* The classifier optimisation MLE process took', classifier.n_iter_[0], 'iterations to converge')
print('* The intercept of the Logistic Regression is:',classifier.intercept_[0])
print('* And the coefficients for each feature are:',classifier.coef_[0])

In [ ]:
# Just a small utility to print the sigmoid function for you guys
def print_sigmoid_function(classifier):
    formula = str(round(classifier.intercept_[0], 2))
    for coef, column in zip(classifier.coef_[0], features):
        formula += ' + (' + str(round(coef, 2)) + '*' + column +')'
    print('The expit function, used to transform the odds into class probabilities, is: type=1/(1+e^(-F))')
    print('Where F is: F=',formula)

print_sigmoid_function(classifier)

In [ ]:
preds = classifier.predict(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

In [ ]:
print('The classifier\'s predictions over the testing data are:\n',list(preds))

In [ ]:
print('And the real testing data we were expecting is:\n',list(test['type']))

In [ ]:
# It is behaving pretty good already:
right = 0
for real, pred in zip(test['type'], preds):  # With ZIP we join two arrays together, element by element, like a zip
    if real == pred:
        right += 1
print('Correctly guessed:', right, '/',len(preds),'(', round(100*right/len(preds), 2), '%)')

In [ ]:
# Using the model's coefficients and intercept, we can calculate ourselves the probability
# for each test record to belong to one class or another.
types = []

test_values = test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values

for r in test_values:
    t = classifier.intercept_[0] + sum(val*coef for val, coef in zip(r, classifier.coef_[0]))
    p = 1/(1+math.exp(-t))
    types.append(round(p, 4))

prob_reals = list(zip(types, test['type']))



# The previous calculation we did using the sigmoid function (aka expit function to get the actual class
# probabilities) is also implemented in sKlearn using:
NUM_PROB_PREDS_TO_PRINT = 10

print('''In these tuples, the first value is the probability of belonging to one class, and the second 
is the real class we expected. Low probabilities (up to 0.5) are predicted to one class, while high 
probabilities are predicted to another class.\n\n''')

print('We print the first', NUM_PROB_PREDS_TO_PRINT,'class probabilities we calculated along with the real class we were expecting:')
print(prob_reals[:NUM_PROB_PREDS_TO_PRINT])

In [ ]:
print('When sklearn calculates the probabilities, it does for the two classes: The 2 values in each prediction sum 1. The second value is what we calculated above:')
pred_probs = classifier.predict_proba(test[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
print('Printing the first',NUM_PROB_PREDS_TO_PRINT,'values:')
print(pred_probs[:NUM_PROB_PREDS_TO_PRINT])

In [ ]:
# Now I defined a function that plots the 2 classes selected, and the threshold by which Sklearn decides to which
# class each tested data belongs to...
# Below you will see that 0.5 is not necessarily the best threshold, and we can set some other to achieve better
# accuracy.
def plot_prediction_probs_with_real_class(prob_reals, PROBABILITY_THRESHOLD=None):
    ones = []
    twos = []
    first_class = None
    second_class = None
    for prob, real in prob_reals:
        if not first_class:
            first_class = real
        if not second_class and real != first_class:
            second_class = real
        if real == first_class:
            ones.append(prob)
        else:
            twos.append(prob)

    fig = plt.figure(figsize=(9, 7))
    ax1 = fig.add_subplot(111)

    ax1.scatter(x=range(len(ones)), y=ones, c='red', marker='s', label='Flower type '+str(first_class))
    ax1.scatter(x=range(len(twos)), y=twos, c='blue', marker='o', label='Flower type '+str(second_class))
    items_in_x_axis = max(len(ones), len(twos))
    ax1.plot(range(items_in_x_axis), 
             [0.5]*items_in_x_axis, 
             color='grey', 
             linestyle='--',
             label='Sklearn threshold'
            )
    if PROBABILITY_THRESHOLD:
        ax1.plot(range(items_in_x_axis), 
                 [PROBABILITY_THRESHOLD]*items_in_x_axis, 
                 color='black', 
                 linewidth=1.5,
                 linestyle='--',
                 label='Our new threshold'
                )

    plt.title('We can see how our classifier assigns different probabilites to each class, differentiating them:')
    plt.legend(loc='center right', bbox_to_anchor=(1.3, 0.5));
    plt.ylim(0, 1)
    plt.show()
    
print('''The most towards the bottom and top all of the probabilities for the records are, the more sure the 
classifier is that it is classifying that record correctly!\n\n''')

print('Correctly guessed:', right, '/',len(preds),'(', round(100*right/len(preds), 2), '%)')
plot_prediction_probs_with_real_class(prob_reals)

In [ ]:
# A threshold of 0.5 is what sklearn is using. Based on the chart below, can you find a
# better threshold for our data? Input it there and see what accuracy we get.
PROBABILITY_THRESHOLD = 0.4

classes = classifier.classes_
right_predictions = 0
for prob, real in prob_reals:
    if prob > PROBABILITY_THRESHOLD:
        prediction=classes[-1]
    else:
        prediction=classes[0]
    if prediction==real:
        right_predictions+=1
print('Correctly guessed with new threshold:', 
      right_predictions, '/',len(preds),'(', round(100*right_predictions/len(preds), 2), '%)')


plot_prediction_probs_with_real_class(prob_reals, PROBABILITY_THRESHOLD)

# 3. We do some basic feature selection

* We do this to see which features are really contributing to guess correctly the flower type class.

* We can use a chi-squared test to see the z-scores and p-values of each feature in relation to the flower type class (basically using the same data that we used for training the model)
* After checking the p-values and z-scores in the cells below, with their comments, we can build another Logistic Regression classifier but using just some of the features, not all of them. If we removed features that were not helping much in the result, the accuracy of this new classifier shouldn't be much worse (maybe even better) than the previous one, only that we are using much less data to build it and therefore it is lighter and faster to build (if our dataset had millions of rows, the speed and computational space factor would be an issue). 
* You can coment out or remove features below and rebuild again the Logistic Regression classifier re-running the cells, and checking how the accuracy is affected.
* Like before, we added a probability threshold that you can tune to maximise the accuracy after checking the results in a scatter chart.

In [ ]:
# We can do some statistical feature selection to see which features explain the target (flower type)
# and which ones don't. We use the Chi-squared test for that:

from sklearn.feature_selection import chi2

scores, pvalues = chi2(train[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']], train['type'])

In [ ]:
# The p-value for sepal_width is very high, indicating that it is the most independent from the class. 
# In other words, it seems like sepal_width is the feature adding the least information to differentiate 
# classes 1 and 2.
list(zip(features, pvalues))

In [ ]:
# NOTE: The samples are being randomised, and in some cases sepal_length is a bit worse and actually giving less 
# info than sepal_width, so check the parameters being printed by the code to follow up... but keep that in mind

# We can also see their z-scores, the sepal_width parameter is not even 1 standard dev away from the average of the 
# class distribution. Usually, variables with over |2| z-score are the relevant ones, or at least |1|.
list(zip(features, scores))

In [ ]:
# Select which features to use to see which ones still keep a good performance so we can simplify the model:
# We removed sepal_width for you. Can we remove some more for the classes you're classifying?
features = [
    'sepal_length', 
#     'sepal_width', 
    'petal_length', 
    'petal_width'
]

In [ ]:
classifier2 = LogisticRegression(
    random_state=0, penalty='l2'
)
classifier2.fit(train[features], train['type'])

In [ ]:
default_preds = classifier2.predict(test[features])
right_predictions = 0
for pred, real in zip(default_preds, test['type']):
    if pred==real:
        right_predictions+=1
print('Correctly guessed with default threshold of 0.5 :', right_predictions, '/',len(default_preds),'(', round(100*right_predictions/len(default_preds), 2), '%)')    

In [ ]:
# Now we predict again but getting the probabilities so we can plot the predictions
preds = classifier2.predict_proba(test[features])

PROBABILITY_THRESHOLD_2 = 0.38

prob_reals_2 = []
classes = classifier.classes_
right_predictions = 0
for probs, real in zip(preds, test['type']):
    prob_reals_2.append((probs[1], real))
    if probs[1] > PROBABILITY_THRESHOLD_2:
        prediction=classes[-1]
    else:
        prediction=classes[0]
    if prediction==real:
        right_predictions+=1
print('Correctly guessed with thresold of ', PROBABILITY_THRESHOLD_2,':', right_predictions, '/',len(preds),'(', round(100*right_predictions/len(preds), 2), '%)')

plot_prediction_probs_with_real_class(prob_reals_2, PROBABILITY_THRESHOLD_2)



## Do you want to do some basic Linear Regression too?

Of course SciKit Learn also has the classic linear regression: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

Seeing how we have trained the Logistic Regression classifier above, could you do the same for Linear Regression just checking the documentation above?

Note that linear regression is a *true* regression method, so it will return a real value when calling its "predict" function. However you can round that value to the nearest class (0, 1 or 2) if you wanted, so you can use it as a classifier, and also measure its accuracy as we did above (number of correctly predicted test samples over size of the test dataset). You can call the same methods that we used for Logistic Regression: `fit` to fit the regression line, and `predict` to test the test dataset and obtain the predictions for the test samples.

Using Linear Regression as a *benchmark test* is a good idea because many times it's considered one of the most quick and basic methods. Also in many occassions, if the data is good enough, a simple Linear Regression might already give us good enough results and therefore we do not need to do anything more complicated.

If you feel very inspired, feel free to use the matplotlib library to plot some other stuff you might think is relevant to show the linear regression model, like the correctly and wrongly classified samples using your fitted linear regression model.

In [ ]:
from sklearn.linear_model import LinearRegression 

In [ ]:
regressor = LinearRegression(
    n_jobs=1,  # Use one processor in this machine for the MLE process
)